In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from skfeature.utility.util import reverse_argsort
class refiefFAlgo:
    def __init__(self,mode=None):
        self.scoreList=[]
        self.wordIndex=[]
        self.mode=mode
    def feature_ranking(self):
            """
            Rank features in descending order according to reliefF score, the higher the reliefF score, the more important the
            feature is
            """
            for scorePerCluster in self.scoreList:
                temp=np.asarray(scorePerCluster) 
                idx = np.argsort(temp, 0)
            #print(idx)
            self.wordIndex.append(idx[::-1])
            #return idx[::-1]
    def score_ranking(self):
            """
            Rank features in descending order according to reliefF score, the higher the reliefF score, the more important the
            feature is
            """
            for scorePerCluster in self.scoreList:
                temp=np.asarray(scorePerCluster) 
                idx = np.argsort(temp, 0)
            #print(idx)
            self.wordIndex.append(idx[::-1])
            #return idx[::-1]
    def reliefF(self,X, y,**kwargs):
        """
        This function implements the reliefF feature selection

        Input
        -----
        X: {numpy array}, shape (n_samples, n_features)
            input data
        y: {numpy array}, shape (n_samples,)
            input class labels
        kwargs: {dictionary}
            parameters of reliefF:
            k: {int}
                choices for the number of neighbors (default k = 5)

        Output
        ------
        score: {numpy array}, shape (n_features,)
            reliefF score for each feature

        Reference
        ---------
        Robnik-Sikonja, Marko et al. "Theoretical and empirical analysis of relieff and rrelieff." Machine Learning 2003.
        Zhao, Zheng et al. "On Similarity Preserving Feature Selection." TKDE 2013.
        """


        if "k" not in list(kwargs.keys()):
            k = 5
        else:
            k = kwargs["k"]
        n_samples, n_features = X.shape

        # calculate pairwise distances between instances
        distance = pairwise_distances(X, metric='manhattan')



        # the number of sampled instances is equal to the number of total instances
        for idx in range(n_samples):
            score = np.zeros(n_features)
            near_hit = []
            near_miss = dict()

            self_fea = X[idx, :]
            c = np.unique(y).tolist()

            stop_dict = dict()
            for label in c:
                stop_dict[label] = 0
            del c[c.index(y[idx])]

            p_dict = dict()
            p_label_idx = float(len(y[y == y[idx]]))/float(n_samples)

            for label in c:
                p_label_c = float(len(y[y == label]))/float(n_samples)
                p_dict[label] = p_label_c/(1-p_label_idx)
                near_miss[label] = []

            distance_sort = []
            distance[idx, idx] = np.max(distance[idx, :])

            for i in range(n_samples):
                distance_sort.append([distance[idx, i], int(i), y[i]])
            distance_sort.sort(key=lambda x: x[0])

            for i in range(n_samples):
                # find k nearest hit points
                if distance_sort[i][2] == y[idx]:
                    if len(near_hit) < k:
                        near_hit.append(distance_sort[i][1])
                    elif len(near_hit) == k:
                        stop_dict[y[idx]] = 1
                else:
                    # find k nearest miss points for each label
                    if len(near_miss[distance_sort[i][2]]) < k:
                        near_miss[distance_sort[i][2]].append(distance_sort[i][1])
                    else:
                        if len(near_miss[distance_sort[i][2]]) == k:
                            stop_dict[distance_sort[i][2]] = 1
                stop = True
                for (key, value) in list(stop_dict.items()):
                        if value != 1:
                            stop = False
                if stop:
                    break

            # update reliefF score
            near_hit_term = np.zeros(n_features)
            for ele in near_hit:
                near_hit_term = np.array(abs(self_fea-X[ele, :]))+np.array(near_hit_term)

            near_miss_term = dict()
            for (label, miss_list) in list(near_miss.items()):
                near_miss_term[label] = np.zeros(n_features)
                for ele in miss_list:
                    near_miss_term[label] = np.array(abs(self_fea-X[ele, :]))+np.array(near_miss_term[label])
                score += near_miss_term[label]/(k*p_dict[label])
            score -= near_hit_term/k
            self.scoreList.append(score)
            self.feature_ranking()
        if self.mode == 'raw':
            print("here")
            print(score)
            return score
        elif self.mode == 'index':
            print("herew")
            return feature_ranking(score)
        elif self.mode == 'rank':
            print("hereq")
            return reverse_argsort(feature_ranking(score), X.shape[1])


